In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpad

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Creamos el Spark Context

In [5]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
type(sc)

pyspark.context.SparkContext

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
DATA_PATH = "/content/drive/MyDrive/CienciaDeDatos/TP1/data/orders.csv"

sqlContext = SQLContext(sc)
df = sqlContext.read.csv(DATA_PATH, header=True, inferSchema=True)

selected_columns_df = df.select("shipping_address", "discount_amount", "total_amount", "currency")
rdd = selected_columns_df.rdd

rdd = rdd.filter(lambda row: row["shipping_address"] is not None and row["discount_amount"] is not None)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# 3) Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.

In [9]:
import pandas as pd

from google.colab import drive
sqlContext = SQLContext(sc)

drive.mount("/content/drive")
DATA_PATH = "/content/drive/MyDrive/CienciaDeDatos/TP1/data/"

rdd_orders = (
    sqlContext.read.csv(DATA_PATH + 'orders.csv', header=True, inferSchema=True)
    .select("payment_method", "customer_id")
    .rdd
)

rdd_customers = (
    sqlContext.read.csv(DATA_PATH + 'customers.csv', header=True, inferSchema=True)
    .select('customer_id', 'customer_segment','is_active','marketing_consent')
    .rdd
)

TOP_CPS_SHOWED = 5
TOP_NAMES_SHOWED = 1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from pyspark.sql import SQLContext, functions as F
sqlContext = SQLContext(sc)

# ------------------- Helpers -------------------
def normalize_category(value):
    if value is None:
        return None
    v = str(value).strip().lower()
    if v in {"nan", "na", "undefined", "none", ""}:
        return None
    return v.title()

def to_int_bool(v):
    if v is None:
        return 0
    if isinstance(v, bool):
        return 1 if v else 0
    s = str(v).strip().lower()
    if s in {"true", "1", "si", "sí", "y", "yes"}:
        return 1
    if s in {"false", "0", "no", "n"}:
        return 0
    try:
        return 1 if int(float(s)) != 0 else 0
    except:
        return 0
# ------------------- Normalizaciones -------------------
rdd_orders_nm = (
    rdd_orders
    .map(lambda r: (r["customer_id"], normalize_category(r["payment_method"])))
    .filter(lambda kv: kv[0] is not None and kv[1] is not None)
    .distinct()
)

rdd_customers_nm = (
    rdd_customers
    .map(lambda r: (
        r["customer_id"],
        (
            normalize_category(r["customer_segment"]),
            to_int_bool(r["is_active"]),
            to_int_bool(r["marketing_consent"])
        )
    ))
    .filter(lambda kv: kv[0] is not None and kv[1][0] is not None)
)

# (customer_id, ((segmento, activo, consent), tipo_pago))
rdd_join = rdd_customers_nm.join(rdd_orders_nm)

# Key: (tipo_pago, segmento)
# Value: (suma_activos, suma_consentimiento, total_clientes)
rdd_kv = rdd_join.map(lambda item: (
    (item[1][1], item[1][0][0]),
    (item[1][0][1], item[1][0][2], 1)
))

rdd_sums = rdd_kv.reduceByKey(lambda a, b: (
    a[0] + b[0],   # activos
    a[1] + b[1],   # consentimiento
    a[2] + b[2]    # total
))

rdd_result = rdd_sums.map(lambda kv: {
    "tipo_pago": kv[0][0],
    "segmento": kv[0][1],
    "clientes_totales": kv[1][2],
    "clientes_activos_suma": kv[1][0],
    "clientes_activos_promedio_%": 100.0 * kv[1][0] / kv[1][2] if kv[1][2] else 0.0,
    "consentimiento_marketing_suma": kv[1][1],
    "consentimiento_marketing_promedio_%": 100.0 * kv[1][1] / kv[1][2] if kv[1][2] else 0.0
})

# ------------------- Pivots -- -------------------
df_result = sqlContext.createDataFrame(rdd_result)


pivot_activos = (
    df_result
    .groupBy("tipo_pago")
    .pivot("segmento")
    .agg(F.first("clientes_activos_promedio_%"))
    .orderBy("tipo_pago")
)


pivot_consent = (
    df_result
    .groupBy("tipo_pago")
    .pivot("segmento")
    .agg(F.first("consentimiento_marketing_promedio_%"))
    .orderBy("tipo_pago")
)

print("----- Porcentajes de Clientes Activos -----")
pivot_activos.show(truncate=False)

print("----- Porcentajes de Consentimiento Marketing -----")
pivot_consent.show(truncate=False)


----- Porcentajes de Clientes Activos -----
+----------------+-----------------+-----------------+-----------------+
|tipo_pago       |Budget           |Premium          |Regular          |
+----------------+-----------------+-----------------+-----------------+
|Bank Transfer   |89.78004141713774|89.88082221736278|89.96447276856023|
|Cash On Delivery|89.79454738845659|89.87860892388451|89.96704503844745|
|Credit Card     |89.77889728519452|89.87203324948048|89.96228073387776|
|Debit Card      |89.77603583426652|89.88628908812596|89.97179797091894|
|Digital Wallet  |89.78894922465432|89.88518316019683|89.96630406914991|
|Paypal          |89.78894922465432|89.87750191403259|89.96338337605273|
+----------------+-----------------+-----------------+-----------------+

----- Porcentajes de Consentimiento Marketing -----
+----------------+-----------------+-----------------+-----------------+
|tipo_pago       |Budget           |Premium          |Regular          |
+----------------+---------

# Conclusiones:

Se observó una sorpresivamente baja variabilidad de porcentajes entre métodos y segmentos: la tasa de clientes activos se ubica en torno al 89,8–90,0% para todos los clientes y la de consentimiento de marketing alrededor del 69,7–70,1%.
A partir de eso se concluye que el segmento de cliente y el método de pago de elección a la hora de realizar una orden, no tienen correlación con la probabilidad de que el cliente sea activo o que consienta al acuerdo de marketing.
Fuera de esto, sí se observó una sola distinción entre los valores y es que hay muchos más clientes en el segmento Regular que en los otros segmentos. Se puede pensar que esto es debido a que el servicio probablemente sea pago.